### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=IMPORT PACKAGES=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [5]:
import os
import requests
import sqlite3
import numpy as np
from scipy.signal import find_peaks
import pandas as pd

from sklearn.pipeline import *
from sklearn.preprocessing import *
from sklearn.neighbors import *
from sklearn.model_selection import *
from sklearn.feature_selection import *
from sklearn.metrics import *
from sklearn.base import *
from sklearn.svm import *
from sklearn.ensemble import *




### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=FILE DOWNLOAD=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [1]:
#Initilizes Files and Header Information
file_list  = [
    "SiCo30_01.txt", "GaCo11_01.txt", "SiCo27_01.txt", "SiCo14_01.txt", "SiCo17_01.txt", "GaCo02_01.txt", "GaCo13_01.txt", "GaCo17_01.txt", "GaCo08_01.txt", "SiCo18_01.txt",
    "SiCo06_01.txt", "GaCo12_01.txt", "SiCo03_01.txt", "GaCo06_01.txt", "SiCo19_01.txt", "GaCo04_01.txt", "GaCo22_01.txt", "SiCo04_01.txt", "SiCo25_01.txt", "GaCo16_01.txt",
    "SiCo10_01.txt", "SiCo20_01.txt", "SiCo21_01.txt", "GaCo09_01.txt", "SiCo13_01.txt", "SiCo09_01.txt", "GaCo01_01.txt", "GaCo05_01.txt", "SiCo26_01.txt", "GaCo10_01.txt",
    "SiCo12_01.txt", "SiCo29_01.txt", "SiCo15_01.txt", "SiCo11_01.txt", "SiCo28_01.txt", "SiCo07_01.txt", "SiCo24_01.txt", "SiCo01_01.txt", "GaCo03_01.txt", "GaCo15_01.txt",
    "SiPt34_01.txt", "GaPt13_01.txt", "GaPt14_01.txt", "SiPt13_01.txt", "SiPt38_01.txt", "SiPt21_01.txt", "SiPt02_01.txt", "SiPt07_01.txt", "GaPt03_01.txt", "GaPt30_01.txt",
    "SiPt36_01.txt", "SiPt28_01.txt", "GaPt07_01.txt", "GaPt19_01.txt", "SiPt04_01.txt", "SiPt18_01.txt", "GaPt23_01.txt", "GaPt15_01.txt", "GaPt20_01.txt", "GaPt12_01.txt",
    "SiPt39_01.txt", "SiPt32_01.txt", "SiPt30_01.txt", "GaPt22_01.txt", "SiPt40_01.txt", "GaPt05_01.txt", "GaPt33_01.txt", "SiPt33_01.txt", "SiPt10_01.txt", "GaPt08_01.txt",
    "GaPt24_01.txt", "SiPt05_01.txt", "GaPt04_01.txt", "SiPt24_01.txt", "SiPt35_01.txt", "SiPt23_01.txt", "SiPt08_01.txt", "SiPt25_01.txt", "SiPt12_01.txt", "GaPt27_01.txt"
]
cols = [
    "time", "sensor_l1", "sensor_l2", "sensor_l3", "sensor_l4", "sensor_l5",
    "sensor_l6", "sensor_l7", "sensor_l8", "sensor_r1", "sensor_r2", "sensor_r3",
    "sensor_r4", "sensor_r5", "sensor_r6", "sensor_r7", "sensor_r8", "total_left", "total_right"
]
# Gets a List of all Patient Names by Cleaning Up file Names
patients = [os.path.splitext(f)[0].replace("_01", "") for f in file_list]


base_url = "https://physionet.org/files/gaitpdb/1.0.0/"
dst_dir = r"Data/RawData/"
os.makedirs(dst_dir, exist_ok=True)

# Code to Download files
for fname in file_list:
    url = base_url + fname+ "?download"
    dst_path = os.path.join(dst_dir, fname)

    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(dst_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded {fname} from {url} to {dst_path}")
    except requests.HTTPError as e:
        print(f"Failed to download {fname}: {e}")

Downloaded SiCo30_01.txt from https://physionet.org/files/gaitpdb/1.0.0/SiCo30_01.txt?download to Data/RawData/SiCo30_01.txt
Downloaded GaCo11_01.txt from https://physionet.org/files/gaitpdb/1.0.0/GaCo11_01.txt?download to Data/RawData/GaCo11_01.txt
Downloaded SiCo27_01.txt from https://physionet.org/files/gaitpdb/1.0.0/SiCo27_01.txt?download to Data/RawData/SiCo27_01.txt
Downloaded SiCo14_01.txt from https://physionet.org/files/gaitpdb/1.0.0/SiCo14_01.txt?download to Data/RawData/SiCo14_01.txt
Downloaded SiCo17_01.txt from https://physionet.org/files/gaitpdb/1.0.0/SiCo17_01.txt?download to Data/RawData/SiCo17_01.txt
Downloaded GaCo02_01.txt from https://physionet.org/files/gaitpdb/1.0.0/GaCo02_01.txt?download to Data/RawData/GaCo02_01.txt
Downloaded GaCo13_01.txt from https://physionet.org/files/gaitpdb/1.0.0/GaCo13_01.txt?download to Data/RawData/GaCo13_01.txt
Downloaded GaCo17_01.txt from https://physionet.org/files/gaitpdb/1.0.0/GaCo17_01.txt?download to Data/RawData/GaCo17_01.txt


### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=DATA STORING IN DATABASE=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [21]:
# Creates a SQLite Database so that data can be tracked maintained and manipulated slightly easier then in Massive Arrays
db_file = "Gait_Database.sqlite"
conn = sqlite3.connect(db_file)
cur = conn.cursor()

#Reads Demographic Data Into Database
df = pd.read_excel("./Data/demographics.xls" )
first_col = df.columns[0]
df = df[df[first_col].isin(patients)]
df.to_sql("demographics", conn, if_exists="replace", index=False)

# Reads all patient data in to Database
for i, file in enumerate(files):
    df = pd.read_csv(f"./Data/RawData/{file}", sep="\t", header=None)
    df.columns = headers
    df.to_sql(patients[i], conn, if_exists="replace", index=False)

# Dont actually need the time data, so we can remove this
cols.pop(0)


### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=DATA IMPUTATION=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [10]:
# SQL command that fills in any "NaN" with the average based on Gender
for col in ["Height (meters)", "Weight (kg)", "Speed_01 (m/sec)", "TUAG"]:
    cur.execute(f"""
        UPDATE demographics AS d
        SET "{col}" = (
            SELECT ROUND(AVG(d2."{col}"), 2)
            FROM demographics AS d2
            WHERE (LOWER(d2.Gender)='male') = (LOWER(d.Gender)='male')
        )
        WHERE "{col}" IS NULL;
    """)
    print(f"Filled missing values for {col}")

#Commit Change to Database
conn.commit()

Filled missing values for Height (meters)
Filled missing values for Weight (kg)
Filled missing values for Speed_01 (m/sec)
Filled missing values for TUAG


### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=Feature Extraction=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [20]:
fs = 100  # Hz

#Method that will print a summary of the Features (Mean, min, max, std, etc)
#Prints in LaTeX table form so I can copy it into report directly if I make any changes
#Is a function because I need to do this twice- once before scaling, and once after
def print_feature_stats(features, feature_names):
    print("\n=== Feature Summary Statistics ===")
    print("Feature & Min & Max & Mean & Std & Median \\\\")
    print("---------------------------------------------")
    for name, col in zip(feature_names, features.T):
        print(f"{name} & {np.min(col):.2f} & {np.max(col):.2f} & "
              f"{np.mean(col):.2f} & {np.std(col, ddof=1):.2f} & {np.median(col):.2f}\\\\")
    print("---------------------------------------------\n")

#Representations of Left and Right Toes/Heels
L_HEEL, L_TOE = ["sensor_l1","sensor_l2","sensor_l3"], ["sensor_l6","sensor_l7","sensor_l8"]
R_HEEL, R_TOE = ["sensor_r1","sensor_r2","sensor_r3"], ["sensor_r6","sensor_r7","sensor_r8"]

#Initilize Features and Classiciations
features = np.zeros((len(patients), 15), float)
classification = np.zeros(len(patients), int)

#For Each Patient:
for i, pid in enumerate(patients):

    #get their data and turn it into a dict called S
    cur.execute(f'SELECT {", ".join(cols)} FROM "{pid}"')
    data = np.asarray(cur.fetchall(), float)
    S = {c: data[:, j] for j, c in enumerate(cols)}
    gender, age, speed, height, TUAG = cur.execute(
    'SELECT Gender, Age, "Speed_01 (m/sec)", "Height (meters)", "TUAG" FROM demographics WHERE ID=?',
    (pid,)
    ).fetchone()

    #Step Calculations
    total_left, total_right = S["total_left"], S["total_right"]
    steps_left,  _ = find_peaks(total_left,  prominence=400, height=0.5*np.max(total_left))
    steps_right, _ = find_peaks(total_right, prominence=400, height=0.5*np.max(total_right))

    #Stride Calculation
    stride_left, stride_right = np.diff(steps_left)/fs, np.diff(steps_right)/fs
    mean_stride_L, mean_stride_R = np.mean(stride_left), np.mean(stride_right)
    stride_all = np.concatenate([stride_left, stride_right])

    #Stance Calculation
    stance_L, stance_R = (heel_L + toe_L) > 0, (heel_R + toe_R) > 0
    stance_pct_L = np.array([100*np.mean(stance_L[a:b]) for a,b in zip(steps_left[:-1], steps_left[1:])])
    stance_pct_R = np.array([100*np.mean(stance_R[a:b]) for a,b in zip(steps_right[:-1], steps_right[1:])])
    mean_stance_L, mean_stance_R = np.mean(stance_pct_L), np.mean(stance_pct_R)

    #Swing Calculation
    swing_pct_L, swing_pct_R  = 100 - stance_pct_L, 100 - stance_pct_R
    mean_swing_L, mean_swing_R = np.mean(swing_pct_L), np.mean(swing_pct_R)
    swing_all  = np.concatenate([swing_pct_L, swing_pct_R])

    #Step Calculation
    idx_R_after_L = np.searchsorted(steps_right, steps_left)
    valid_LR = idx_R_after_L < len(steps_right)
    step_LR = (steps_right[idx_R_after_L[valid_LR]] - steps_left[valid_LR]) / fs
    idx_L_after_R = np.searchsorted(steps_left, steps_right)
    valid_RL = idx_L_after_R < len(steps_left)
    step_RL = (steps_left[idx_L_after_R[valid_RL]] - steps_right[valid_RL]) / fs
    mean_step_LR, mean_step_RL = np.mean(step_LR), np.mean(step_RL)
    step_all   = np.concatenate([step_LR, step_RL])

    #Heel-Toe Calculation
    heel_L = np.mean(np.vstack([S[k] for k in L_HEEL]), axis=0)
    toe_L  = np.mean(np.vstack([S[k] for k in L_TOE ]), axis=0)
    heel_R = np.mean(np.vstack([S[k] for k in R_HEEL]), axis=0)
    toe_R  = np.mean(np.vstack([S[k] for k in R_TOE ]), axis=0)
    htr_L = np.mean(heel_L[stance_L]) / np.mean(toe_L[stance_L])
    htr_R = np.mean(heel_R[stance_R]) / np.mean(toe_R[stance_R])

    #Double Support Calculation
    thrL, thrR = 0.02*np.max(total_left), 0.02*np.max(total_right)
    contact_L, contact_R = total_left > thrL, total_right > thrR
    n_min = min(contact_L.size, contact_R.size)
    double_support = np.mean(contact_L[:n_min] & contact_R[:n_min])

    features[i] = [
        np.std(stride_all, ddof=1) / np.mean(stride_all),   # 0  Stride Time Var.
        np.std(swing_all,  ddof=1) / np.mean(swing_all),    # 1  Swing Time Var.
        np.std(step_all,   ddof=1) / np.mean(step_all),     # 2  Step Time Var.
        mean_stride_L / mean_stride_R,                      # 3  Stride Time Asym.
        mean_swing_L / mean_swing_R,                        # 4  Swing Time Asym.
        mean_step_LR / mean_step_RL,                        # 5  Step Time Asym.
        mean_stance_L / mean_stance_R,                      # 6  Stance Time Asym.
        double_support,                                     # 7  Double Support
        0.5*(htr_L + htr_R),                                # 8  Heel-Toe Pressure
        htr_L / htr_R,                                      # 9  Heel-Toe Pressure Asym.
        1.0 if gender.lower() == "male" else 0.0,           # 10 Gender
        float(age),                                         # 11 Age
        float(speed),                                       # 12 Speed
        float(height),                                      # 13 Height
        float(TUAG)                                         # 14 TUAG
    ]

    classification[i] = 1 if "Pt" in pid else 0

feature_names = np.array([
    "Stride Time Var.",        "Swing Time Var.",    "Step Time Var.",
    "Stride Time Asym.",       "Swing Time Asym.",   "Step Time Asym.",
    "Stance Time Asym.",       "Double Support",     "Heel-Toe Pressure",
    "Heel-Toe Pressure Asym.", "Gender",             "Age",
    "Speed",                   "Height",             "TUAG"
])

print_feature_stats(features, feature_names)



=== Feature Summary Statistics ===
Feature & Min & Max & Mean & Std & Median \\
---------------------------------------------
Stride Time Var. & 0.03 & 0.55 & 0.13 & 0.08 & 0.12\\
Swing Time Var. & 0.09 & 0.64 & 0.28 & 0.12 & 0.25\\
Step Time Var. & 0.04 & 0.77 & 0.22 & 0.15 & 0.17\\
Stride Time Asym. & 0.86 & 1.17 & 1.00 & 0.03 & 1.00\\
Swing Time Asym. & 0.68 & 1.37 & 0.99 & 0.12 & 0.99\\
Step Time Asym. & 0.36 & 3.48 & 1.08 & 0.41 & 1.02\\
Stance Time Asym. & 0.89 & 1.12 & 1.00 & 0.04 & 1.00\\
Double Support & 0.21 & 0.55 & 0.32 & 0.06 & 0.30\\
Heel-Toe Pressure & 0.47 & 2.20 & 1.05 & 0.36 & 1.00\\
Heel-Toe Pressure Asym. & 0.24 & 2.85 & 1.11 & 0.44 & 1.10\\
Gender & 0.00 & 1.00 & 0.59 & 0.50 & 1.00\\
Age & 36.00 & 86.00 & 64.53 & 9.88 & 65.00\\
Speed & 0.36 & 1.54 & 1.14 & 0.23 & 1.15\\
Height & 1.45 & 1.95 & 1.68 & 0.09 & 1.68\\
TUAG & 6.23 & 36.34 & 11.06 & 3.89 & 10.46\\
---------------------------------------------



### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=Feature Re-Scaling=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [23]:
#Applies Re-scaling to Features
features = MinMaxScaler().fit_transform(features)
print_feature_stats(features, feature_names)


=== Feature Summary Statistics ===
Feature & Min & Max & Mean & Std & Median \\
---------------------------------------------
Stride Time Var. & 0.00 & 1.00 & 0.19 & 0.16 & 0.18\\
Swing Time Var. & 0.00 & 1.00 & 0.34 & 0.21 & 0.29\\
Step Time Var. & 0.00 & 1.00 & 0.24 & 0.21 & 0.17\\
Stride Time Asym. & 0.00 & 1.00 & 0.45 & 0.11 & 0.45\\
Swing Time Asym. & 0.00 & 1.00 & 0.46 & 0.18 & 0.45\\
Step Time Asym. & 0.00 & 1.00 & 0.23 & 0.13 & 0.21\\
Stance Time Asym. & 0.00 & 1.00 & 0.49 & 0.18 & 0.49\\
Double Support & 0.00 & 1.00 & 0.31 & 0.18 & 0.27\\
Heel-Toe Pressure & 0.00 & 1.00 & 0.34 & 0.21 & 0.31\\
Heel-Toe Pressure Asym. & 0.00 & 1.00 & 0.33 & 0.17 & 0.33\\
Gender & 0.00 & 1.00 & 0.59 & 0.50 & 1.00\\
Age & 0.00 & 1.00 & 0.57 & 0.20 & 0.58\\
Speed & 0.00 & 1.00 & 0.66 & 0.19 & 0.67\\
Height & 0.00 & 1.00 & 0.45 & 0.18 & 0.46\\
TUAG & 0.00 & 1.00 & 0.16 & 0.13 & 0.14\\
---------------------------------------------



### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=Feature Re-Scaling=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [32]:
"""
My KNN function- for reproducability between code and report, I use my student number as the random state. I do cross validation both for KNN and SFS, and
so I use my student number +1 for the second cross-validation to ensure randomness. I use ROC-auc as the scoring metric, as that is the metric I care most about.
I use stratified Kfolds to ensure some level of balance between folds, due to the small dataset and high variance.
"""

def knn(X, y, K, weights="uniform"):
    #Initilizes Folds folds for KNN and SFS
    KNN_folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=21207103)
    SFS_folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=21207103+1)
    base_knn = KNeighborsClassifier(n_neighbors=K, weights=weights)
    metrics, fold_details = [], []

    #Loop representing each KNN Fold
    for tr, te in KNN_folds.split(X, y):
        #Initilize SFS, ensuring that estimator is the same as what will model (as a clone to avoid leakage
        sfs = SequentialFeatureSelector(
            estimator=clone(base_knn),
            n_features_to_select="auto",
            direction="forward",
            scoring="roc_auc",
            tol=1e-5,
            cv=SFS_folds,
            n_jobs=-1
            )
        #applying sfs and fitting
        pipe = Pipeline([
            ("sfs", clone(sfs)),
            ("knn", clone(base_knn))
        ])
        pipe.fit(X[tr], y[tr])

        #testing and getting results
        yp = pipe.predict(X[te])
        acc = accuracy_score(y[te], yp)
        f1m = f1_score(y[te], yp, average="macro")
        auc = roc_auc_score(y[te], pipe.predict_proba(X[te])[:, 1])
        n_selected = np.sum(pipe.named_steps["sfs"].get_support())

        #appending rsults for return
        fold_details.append([acc, f1m, auc, n_selected])
        metrics.append([acc, f1m, auc])

    return np.array(metrics), fold_details

#Printing functions for visibility
def print_knn(scores, K):
    mean, std = np.nanmean(scores, axis=0), np.nanstd(scores, axis=0)
    print(
        f"{K} & "
        f"{mean[0]:.2f} & {std[0]:.2f} & "
        f"{mean[1]:.2f} & {std[1]:.2f} & "
        f"{mean[2]:.2f} & {std[2]:.2f}\\\\"
    )

#Printing functions for visibility
def print_knn_folds(fold_details, K, weights):
    print(f"Per-fold results (K={K}, weights={weights}):")
    print("Fold & Acc & F1 & AUC & #Sel\\\\")
    for acc, f1m, auc, nsel in fold_details:
        print(f"{acc:.2f} & {f1m:.2f} & {auc:.2f} & {int(nsel)}\\\\")


In [47]:
results = {}
print("K  & Mean Acc. & Acc. Std. Dev. & Mean F1 & F1 Std. Dev. & Mean AUC & AUC Std. Dev")
for K in [1, 3, 5, 7, 9]:
    scores, folds = knn(features, classification, K=K, weights="uniform")
    print_knn(scores, K)     # table line
    results[K] = (scores, folds)

# --- after summary table ---
print("\nDetailed per-fold results:")
for K, (scores, folds) in results.items():
    print_knn_folds(folds, K, "uniform")
    print()  # blank line between K-values

K  & Mean Acc. & Acc. Std. Dev. & Mean F1 & F1 Std. Dev. & Mean AUC & AUC Std. Dev
1 & 0.64 & 0.10 & 0.63 & 0.10 & 0.64 & 0.10\\
3 & 0.62 & 0.10 & 0.61 & 0.10 & 0.65 & 0.12\\
5 & 0.61 & 0.14 & 0.60 & 0.14 & 0.72 & 0.11\\
7 & 0.71 & 0.19 & 0.70 & 0.19 & 0.81 & 0.16\\
9 & 0.61 & 0.17 & 0.59 & 0.19 & 0.73 & 0.16\\

Detailed per-fold results:
Per-fold results (K=1, weights=uniform):
Fold & Acc & F1 & AUC & #Sel\\
79 & 0.62 & 0.62 & 0.62 & 4\\
79 & 0.62 & 0.62 & 0.62 & 5\\
79 & 0.88 & 0.87 & 0.88 & 6\\
79 & 0.62 & 0.62 & 0.62 & 4\\
79 & 0.62 & 0.62 & 0.62 & 4\\
79 & 0.62 & 0.62 & 0.62 & 6\\
79 & 0.50 & 0.50 & 0.50 & 3\\
79 & 0.62 & 0.62 & 0.62 & 2\\
79 & 0.75 & 0.73 & 0.75 & 6\\
79 & 0.50 & 0.50 & 0.50 & 3\\

Per-fold results (K=3, weights=uniform):
Fold & Acc & F1 & AUC & #Sel\\
79 & 0.75 & 0.73 & 0.75 & 4\\
79 & 0.75 & 0.73 & 0.62 & 2\\
79 & 0.75 & 0.73 & 0.75 & 4\\
79 & 0.62 & 0.62 & 0.59 & 2\\
79 & 0.62 & 0.62 & 0.53 & 2\\
79 & 0.62 & 0.62 & 0.66 & 7\\
79 & 0.50 & 0.50 & 0.44 & 2\\
79 &

In [48]:
results = {}
print("K  & Mean Acc. & Acc. Std. Dev. & Mean F1 & F1 Std. Dev. & Mean AUC & AUC Std. Dev")
for K in [1, 3, 5, 7, 9]:
    scores, folds = knn(features, classification, K=K, weights="distance")
    print_knn(scores, K)     # table line
    results[K] = (scores, folds)

# --- after summary table ---
print("\nDetailed per-fold results:")
for K, (scores, folds) in results.items():
    print_knn_folds(folds, K, "distance")
    print()  # blank line between K-values

K  & Mean Acc. & Acc. Std. Dev. & Mean F1 & F1 Std. Dev. & Mean AUC & AUC Std. Dev
1 & 0.64 & 0.10 & 0.63 & 0.10 & 0.64 & 0.10\\
3 & 0.70 & 0.15 & 0.69 & 0.16 & 0.71 & 0.17\\
5 & 0.70 & 0.15 & 0.69 & 0.16 & 0.78 & 0.16\\
7 & 0.57 & 0.18 & 0.56 & 0.19 & 0.68 & 0.20\\
9 & 0.64 & 0.21 & 0.63 & 0.22 & 0.68 & 0.23\\

Detailed per-fold results:
Per-fold results (K=1, weights=distance):
Fold & Acc & F1 & AUC & #Sel\\
79 & 0.62 & 0.62 & 0.62 & 4\\
79 & 0.62 & 0.62 & 0.62 & 5\\
79 & 0.88 & 0.87 & 0.88 & 6\\
79 & 0.62 & 0.62 & 0.62 & 4\\
79 & 0.62 & 0.62 & 0.62 & 4\\
79 & 0.62 & 0.62 & 0.62 & 6\\
79 & 0.50 & 0.50 & 0.50 & 3\\
79 & 0.62 & 0.62 & 0.62 & 2\\
79 & 0.75 & 0.73 & 0.75 & 6\\
79 & 0.50 & 0.50 & 0.50 & 3\\

Per-fold results (K=3, weights=distance):
Fold & Acc & F1 & AUC & #Sel\\
79 & 0.75 & 0.73 & 0.62 & 4\\
79 & 0.75 & 0.73 & 0.81 & 7\\
79 & 0.62 & 0.56 & 0.62 & 6\\
79 & 0.88 & 0.87 & 0.88 & 5\\
79 & 0.75 & 0.75 & 0.75 & 2\\
79 & 0.88 & 0.87 & 0.81 & 5\\
79 & 0.50 & 0.50 & 0.41 & 2\\
79

### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=Feature Re-Scaling=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [38]:
"""
My SVM function- again, I use my student number as the random state for reproducabiltiy between the two different folds. I again do cross validation both for SVM and SFS. The structure is very similar, except that it uses a SVM rather then KNN. I also have an input to select the value for C and the number of features. I do not use a kernel, as upon testing it did not provide any performance increase, but significantly affected speed
"""

def svm(X, y, C_value, n_features=None):
    #Initlizing folds
    SVMFolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=21207103)
    SFSFolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=21207103+1)

    #Initlize SVM and SFS
    base_svm = SVC(C=C_value)
    sfs = SequentialFeatureSelector(
        estimator=clone(base_svm),
        n_features_to_select="auto" if n_features is None else n_features,
        direction="forward",
        scoring="roc_auc",
        tol=1e-5,
        cv=SFSFolds,
        n_jobs=-1
    )
    metrics = []
    n_selected = []

    #Represent Each SVM fold
    for tr, te in SVMFolds.split(X, y):
        #Pipeline and fitting
        pipe = Pipeline([
            ("sfs", clone(sfs)),
            ("svm", clone(base_svm))
        ])
        pipe.fit(X[tr], y[tr])
        n_selected.append(len(pipe.named_steps["sfs"].get_support(indices=True)))

        #metric Calculation
        yp = pipe.predict(X[te])
        scores = pipe.decision_function(X[te])
        acc = accuracy_score(y[te], yp)
        f1 = f1_score(y[te], yp, average="binary")
        auc = roc_auc_score(y[te], scores)
        metrics.append([acc, f1, auc])

    return np.array(metrics), np.array(n_selected)


#Function to print info in latex form
def print_svm(scores, C, subset_sizes):
    mean, std = np.nanmean(scores, axis=0), np.nanstd(scores, axis=0)
    mean_subset = np.mean(subset_sizes)

    print(
        f"{C} & "
        f"{mean_subset:.2f} & "
        f"{mean[0]:.2f} & {std[0]:.2f} & "
        f"{mean[1]:.2f} & {std[1]:.2f} & "
        f"{mean[2]:.2f} & {std[2]:.2f}\\\\"
    )


### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= SVM AUTO =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

In [49]:
results = []
print("SVM Auto")
print("C  & Mean Feature Ct. & Mean Acc. & Acc. SD & Mean F1 & F1 SD & Mean AUC & AUC SD\\\\")
for C in [0.1, .5, 1, 5, 10]:

    metrics_auto, subset_sizes = svm(features, classification, C)
    print_svm(metrics_auto, C, subset_sizes)
    results.append(subset_sizes)

print("\nPer Fold Feature Selection Sizing")
print(results)


SVM Auto
C  & Mean Feature Ct. & Mean Acc. & Acc. SD & Mean F1 & F1 SD & Mean AUC & AUC SD\\
0.1 & 3.50 & 0.70 & 0.18 & 0.61 & 0.28 & 0.82 & 0.16\\
0.5 & 3.50 & 0.71 & 0.14 & 0.65 & 0.19 & 0.80 & 0.11\\
1 & 4.00 & 0.64 & 0.13 & 0.62 & 0.14 & 0.78 & 0.12\\
5 & 3.00 & 0.74 & 0.12 & 0.74 & 0.11 & 0.82 & 0.10\\
10 & 3.80 & 0.75 & 0.14 & 0.72 & 0.18 & 0.85 & 0.12\\

Per Fold Feature Selection Sizing
[array([3, 3, 3, 2, 3, 3, 4, 4, 5, 5]), array([4, 3, 3, 2, 3, 3, 4, 4, 4, 5]), array([4, 4, 4, 3, 2, 4, 4, 5, 5, 5]), array([3, 3, 3, 3, 2, 4, 4, 3, 2, 3]), array([3, 4, 6, 3, 3, 6, 4, 3, 3, 3])]


### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= SVM Fixed =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

In [50]:
results = []
print("SVM Fixed")
print("C  & Mean Feature Ct. & Mean Acc. & Acc. SD & Mean F1 & F1 SD & Mean AUC & AUC SD\\\\")
for C in [0.1, .5, 1, 5, 10]:
    metrics_fixed, subset_sizes = svm(features, classification, C, n_features=np.max(subset_sizes))
    print_svm(metrics_fixed, C, subset_sizes)
    results.append(subset_sizes)

print("Per Fold Feature Selection Sizing")
print(results)

SVM Fixed
C  & Mean Feature Ct. & Mean Acc. & Acc. SD & Mean F1 & F1 SD & Mean AUC & AUC SD\\
0.1 & 6.00 & 0.66 & 0.13 & 0.62 & 0.13 & 0.81 & 0.15\\
0.5 & 6.00 & 0.64 & 0.13 & 0.61 & 0.15 & 0.77 & 0.15\\
1 & 6.00 & 0.70 & 0.15 & 0.69 & 0.15 & 0.81 & 0.16\\
5 & 6.00 & 0.74 & 0.14 & 0.74 & 0.15 & 0.89 & 0.10\\
10 & 6.00 & 0.71 & 0.08 & 0.65 & 0.13 & 0.85 & 0.14\\
Per Fold Feature Selection Sizing
[array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6])]


### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= Random Forest =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

In [62]:
"""
The final model, which is again quite similar to KNN and SVM. However, this takes a considerable amount of time to run, so I have had to remove the cross validation from the SFS out of time requirements. I still use the fixed seed of my student number, and otherwise the function is structured quite simialr
"""

def rf(X, y, n_trees, n_features=None):
    #fold initilization
    Folds = StratifiedKFold(n_splits=10, shuffle=True)
    metrics, subset_sizes = [], []

    #rf and sfs initlized
    n_features = "auto" if n_features is None else n_features
    base_rf = RandomForestClassifier(n_estimators=n_trees, max_depth=10)
    sfs = SequentialFeatureSelector(estimator=clone(base_rf), n_features_to_select=n_features, tol=1e-5)

    #loop representing each fold of rf
    for tr, te in Folds.split(X, y):

        #model sfsing and fitting
        pipe = Pipeline([
            ("sfs", clone(sfs)),
            ("rf", clone(base_rf))
        ])
        pipe.fit(X[tr], y[tr])
        yp = pipe.predict(X[te])
        proba = pipe.predict_proba(X[te])[:, 1]

        #metric calcualtion and appending
        acc = accuracy_score(y[te], yp)
        f1m = f1_score(y[te], yp, average="macro")
        auc = roc_auc_score(y[te], proba)
        ksel = pipe.named_steps["sfs"].get_support(indices=True).size
        subset_sizes.append(ksel)
        metrics.append([acc, f1m, auc])

    return np.array(metrics), np.array(subset_sizes)



#function to print data in neat way
def print_rf(scores, n_trees, subset_sizes):
    mean, std = np.nanmean(scores, axis=0), np.nanstd(scores, axis=0)
    mean_subset = np.mean(subset_sizes)
    row = (
        f"{n_trees} & "
        f"{mean_subset:.2f} & "
        f"{mean[0]:.2f} & {std[0]:.2f} & "
        f"{mean[1]:.2f} & {std[1]:.2f} & "
        f"{mean[2]:.2f} & {std[2]:.2f}\\\\"
    )
    print(row)



### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= Random Forest With Auto Feature Selection=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

In [63]:
print("Random Forest")
print("Trees  & Mean Feature Ct. & Mean Acc. & Acc. SD & Mean F1 & F1 SD & Mean AUC & AUC SD\\\\")
results = []
for n_trees in [20, 100]:
    # --- 1) AUTO run ---
    scores_auto, sizes_auto = rf(features, classification, n_trees, n_features=None)  # None -> "auto" inside your fn
    print_rf(scores_auto, n_trees, sizes_auto)
    results.append(sizes_auto)

print(results)



Random Forest
Trees  & Mean Feature Ct. & Mean Acc. & Acc. SD & Mean F1 & F1 SD & Mean AUC & AUC SD\\
20 & 3.50 & 0.62 & 0.12 & 0.61 & 0.13 & 0.71 & 0.15\\
100 & 3.40 & 0.69 & 0.10 & 0.68 & 0.11 & 0.75 & 0.15\\
[array([4, 3, 4, 3, 3, 3, 4, 3, 4, 4]), array([2, 3, 5, 2, 3, 4, 3, 4, 3, 5])]


### =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= Random Forest With Fixed Feature Count =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

In [66]:
print("Random Forest")
print("Trees  & Mean Feature Ct. & Mean Acc. & Acc. SD & Mean F1 & F1 SD & Mean AUC & AUC SD\\\\")
results1 = []
for n_trees in [20, 100]:
    scores_kplus, sizes_kplus = rf(features, classification, n_trees, n_features=np.max(results)+1)
    print_rf(scores_kplus, n_trees, sizes_kplus)
    results1.append(sizes_kplus)

print(results1)

Random Forest
Trees  & Mean Feature Ct. & Mean Acc. & Acc. SD & Mean F1 & F1 SD & Mean AUC & AUC SD\\
20 & 6.00 & 0.69 & 0.10 & 0.67 & 0.11 & 0.71 & 0.16\\
100 & 6.00 & 0.70 & 0.16 & 0.68 & 0.17 & 0.78 & 0.16\\
[array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6])]
